In [1]:
# test
# %run './base.ipynb'

## Preprocessing
- impute (various kinds)/dummy values/drop missing values
- outlier removal?
- Scale (standardize)/don't scale 
- feature selection (PCA, etc.)/none

## Models
- Linear Regression (Anna)
- SVR (Moritz)
- GradientBoostingRegressor (David)

## Params
- cv=10
- scoring=['r2', 'neg_mean_absolute_error', 'neg_mean_squared_error']
- Best params wählen nach MSE, aber auch r^2 notieren für Vergleichbarkeit

## AutoML
- 


## Datasets
- Bike sharing (Kaggle) ()
    - https://www.kaggle.com/c/184702-tu-ml-ws-18-bike-sharing#_=_
    - large samples (train = 8690), small dimension (15)
    - attribute characteristics: numeric, date?
- Student performance (Kaggle) (Moritz)
    - https://www.kaggle.com/c/184702-tu-ml-ws-18-student-performance
    - small samples (train = 198), medium dimension (32)
    - attribute characteristics: numeric, categorical 
- Blog feedback (David)
    - https://archive.ics.uci.edu/ml/datasets/BlogFeedback
    - very large samples (60021), large dimension (281)
    - attribute characteristics: numeric
- Forest fires (Anna)
    - https://archive.ics.uci.edu/ml/datasets/Forest+Fires
    - medium samples (513), small dimension (13) 
    - attribute characteristics: numeric, categorical
    
## Steps
- Imports for all Datasets
- functions for all Regressors

## Notes
On MSE being negative:

```
Yes, this is supposed to happen. The actual MSE is simply the positive version of the number you're getting.

The unified scoring API always maximizes the score, so scores which need to be minimized are negated in order for the unified scoring API to work correctly. The score that is returned is therefore negated when it is a score that should be minimized and left positive if it is a score that should be maximized.
```
from https://stackoverflow.com/questions/21443865/scikit-learn-cross-validation-negative-values-with-mean-squared-error, 

also described in https://stackoverflow.com/questions/21050110/sklearn-gridsearchcv-with-pipeline

In [12]:
# Modules

# import modules
import pandas as pd
import numpy as np
import sklearn as sk
import time
from time import strptime
from datetime import datetime
import math
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from matplotlib import cm as cm
 
# general sklearn/preprocessing
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# Linear Regression
from sklearn.linear_model import LinearRegression

# SVR
from sklearn.svm import SVR

# Gradient Boosted Tree
from sklearn.ensemble import GradientBoostingRegressor

# AutoML
from hpsklearn import HyperoptEstimator, svr_linear, gradient_boosting_regression, pca, normalizer, standard_scaler
from hyperopt import hp
from hyperopt import anneal, rand, tpe, mix

# allows to output plots in the notebook
%matplotlib inline

/Users/annanau/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [8]:
# Preprocessing

# scale data
def scale_data(train_data, test_data = pd.DataFrame):
    """Standardize features by removing the mean and scaling to unit variance
    
    Only scales columns with dtype 'float64' and 'int64'."""
    
    scaler = preprocessing.StandardScaler()
  
    # drop excluded cols, select numeric cols from data and cast them to float64
    train = train_data.select_dtypes(['float64', 'int64']).astype('float64')
    test = test_data.select_dtypes(['float64', 'int64']).astype('float64')
    
    train_index, train_columns = train.index, train.columns
    test_index, test_columns = test.index, test.columns
        
    # Fit on training set only.
    scaler.fit(train)

    # Apply transform to both the training set and the test set.
    train_s = pd.DataFrame(scaler.transform(train[train_columns].values), index=train_index, columns=train_columns)
    train_data.update(train_s)
    if (test.empty):
        return (train_data)
    else:
        test_s = pd.DataFrame(scaler.transform(test[test_columns].values), index=test_index, columns=test_columns)
        test_data.update(test_s)
        return (train_data, test_data)

# replace empty strings with nan
def fillspace_nan(data):
    return data.apply(lambda x: x.replace('', np.nan))

# strip whitespaces
def strip(data):
    return data.apply(lambda x: x.str.strip())

# one hot encoding
def one_hot(data, drop_first = True):
    columns = data.select_dtypes(['object']).columns
    return pd.get_dummies(data, columns = columns, drop_first = drop_first)

# PCA
def my_pca(train_data, test_data, n_comp):
    pca = PCA(n_components = n_comp)
    pca.fit(train_data)
    pca_train = pd.DataFrame(pca.transform(train_data))
    pca_test = pd.DataFrame(pca.transform(test_data))
    return (pca_train, pca_test)

In [13]:
# Scoring
def rmse(neg_mean_square_error):
    return math.sqrt(-1.*neg_mean_square_error)

In [6]:
# Linear Regression
def linear_reg(X_train, y_train, X_test=pd.DataFrame(), y_test=pd.DataFrame()):
    # Fit model
    reg = LinearRegression().fit(X_train, y_train)
    print(f'R^2 value for model: {round(reg.score(X_train, y_train), 5)}')
    #print(f'Coeffiecients: {reg.coef_}')
    
    # Predict test, compute metrics
    if((not(X_test.empty)) & (not(y_test.empty))):
        print("Predict:")
        pred = reg.predict(X_test)
        r2 = round(reg.score(X_test, y_test), 5)
        mse = round(math.sqrt(mean_squared_error(y_test, pred)), 5)
        print(f'RMSE: {mse}')
        print(f'R^2 Score: {r2}')
    
    return reg

In [15]:
# Support Vector Regression 
def run_svr(X_train, y_train, cv, param_grid, 
            X_test=pd.DataFrame(), y_test=pd.DataFrame()):
    """Run SVR
    
    Keyword arguments:
    X_train -- train data
    y_train -- train target
    """
    
    print("GridSearch initializing...")
    gs = GridSearchCV(estimator = SVR(), cv=cv, param_grid=param_grid, iid=True,
                      scoring=['r2', 'neg_mean_absolute_error', 'neg_mean_squared_error'], 
                      refit='neg_mean_squared_error')
    
    print("SVR model in training...")
    gs.fit(X_train, y_train)

    print(f'''MSE: {round(-1.*gs.best_score_, 5)}, ''' \
          f'''RMSE: {round(rmse(gs.best_score_), 5)}, ''')
    print(f'''C: {gs.best_estimator_.C}, ''' \
          f'''kernel: {gs.best_estimator_.kernel}, ''' \
          f'''epsilon: {gs.best_estimator_.epsilon}, ''' \
          f'''gamma: {gs.best_estimator_.gamma} ''')
    
    # Predict test, compute metrics
    if((not(X_test.empty)) & (not(y_test.empty))):
        print("Predict:")
        pred = gs.predict(X_test)
        mse = round(math.sqrt(mean_squared_error(y_test, pred)), 5)
        r2 = round(r2_score(y_test, pred), 5)
        print(f'RMSE: {mse}')
        print(f'R^2 Score: {r2}')

    return gs

In [5]:
# Gradient Boosting

def run_boosted_tree(train_data, train_target, test_data, test_target, param_fix, cv, param_grid):
    print("GridSearch initializing...")
    clf = GridSearchCV(estimator = GradientBoostingRegressor(**param_fix), cv = cv, param_grid = param_grid,  
                       scoring = ['r2', 'neg_mean_absolute_error', 'neg_mean_squared_error'], 
                       refit = 'neg_mean_squared_error', iid=True)
    
    print("GradientBoostedRegressor model in training...")
    t0 = time.time()
    clf.fit(train_data, train_target)
    clf_fit = time.time() - t0
    print("GradientBoostedRegressor model selected and fitted in %.3f s\n" % clf_fit)
    
    print(f'''MSE: {round(-1.*clf.best_score_, 5)}, ''' \
          f'''RMSE: {round(rmse(clf.best_score_), 5)}''')
    
    print(f'Best parameters selected by GridSearch: {clf.best_params_}')
    
    return clf

# does not work with GridSearch
def plot_training_deviance(clf, X_test, y_test):
    test_score = np.zeros((clf.best_params_['n_estimators'],), dtype=np.float64)

    for i, y_pred in enumerate(clf.staged_predict(X_test)):
        test_score[i] = clf.loss_(y_test, y_pred)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.title('Deviance')
    plt.plot(np.arange(params['n_estimators']) + 1, clf.train_score_, 'b-',
             label='Training Set Deviance')
    plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
             label='Test Set Deviance')
    plt.legend(loc='upper right')
    plt.xlabel('Boosting Iterations')
    plt.ylabel('Deviance')

# kinda not works as expected
def plot_scores(results):
    scoring = ['r2', 'neg_mean_squared_error']
    
    plt.figure(figsize=(13, 13))
    plt.title("GridSearchCV evaluating using multiple scorers simultaneously",
          fontsize=16)

    plt.xlabel("n_estimators")
    plt.ylabel("Score")

    ax = plt.gca()
    ax.set_xlim(0, 402)
    ax.set_ylim(0, 1)

    # Get the regular numpy array from the MaskedArray
    X_axis = np.array(results['param_n_estimators'].data, dtype=float)

    for scorer, color in zip(sorted(scoring), ['g', 'k']):
        for sample, style in (('train', '--'), ('test', '-')):
            sample_score_mean = results['mean_%s_%s' % (sample, scorer)]
            sample_score_std = results['std_%s_%s' % (sample, scorer)]
            ax.fill_between(X_axis, sample_score_mean - sample_score_std,
                            sample_score_mean + sample_score_std,
                            alpha=0.1 if sample == 'test' else 0, color=color)
            ax.plot(X_axis, sample_score_mean, style, color=color,
                    alpha=1 if sample == 'test' else 0.7,
                    label="%s (%s)" % (scorer, sample))

        best_index = np.nonzero(results['rank_test_%s' % scorer] == 1)[0][0]
        best_score = results['mean_test_%s' % scorer][best_index]

        # Plot a dotted vertical line at the best score for that scorer marked by x
        ax.plot([X_axis[best_index], ] * 2, [0, best_score],
                linestyle='-.', color=color, marker='x', markeredgewidth=3, ms=8)

        # Annotate the best score for that scorer
        ax.annotate("%0.2f" % best_score,
                    (X_axis[best_index], best_score + 0.005))

    plt.legend(loc="best")
    plt.grid('off')
    plt.show()

In [2]:
# AutoML

# Make sure training_frame, X_test, y_test are H2OFrame dataframes 
def run_autoML2(y, training_frame, X_test, y_test):

    # Run AutoML for 20 base models (limited to 1 hour max runtime by default)
    aml = H2OAutoML(max_models=30, max_runtime_secs=300, seed=1)
    aml.train(y=y, training_frame=training_frame)
    
    # View the AutoML Leaderboard
    display('AutoML Leaderboard')
    lb = aml.leaderboard
    print(lb.head(rows=lb.nrows)) # Print all rows instead of default (10 rows)
    
    # Predict test set
    preds = aml.predict(X_test)
    preds_array = preds.as_data_frame().as_matrix() # First convert preds to a pandas df, then to a numpy array
    rmse = round(math.sqrt(mean_squared_error(y_test, preds_array)), 5)
    r2 = round(r2_score(y_test, preds_array), 5)
    print(f'RMSE: {rmse}')
    print(f'R^2 Score: {r2}')
    
    return aml

def run_autoML(train_data, train_target, test_data, test_target, 
               preprocessing_ = {None}, classifier_ = {None}):
    # define set of classifier
    if classifier_ == {None}:
        #classifier_ = hp.choice('clf',[GradientBoostingRegressor(learning_rate = 0.01, loss = 'ls')])
        classifier_ = hp.choice('clf',[
            svr_linear('svr', max_iter=1e5),
            gradient_boosting_regression('gbt', learning_rate = 0.01, loss = 'ls')])
    else:
        classifier_ = classifier_
        
    # define preprocessing options
    if preprocessing_ == {None}:
        preprocessing_ = hp.choice('preproc',
            [pca('pca'), normalizer('norm'),
            standard_scaler('stand')])
    else:
        preprocessing_ = preprocessing_
    
    print("AutoML estimator initializing...")
    estim = HyperoptEstimator(
        regressor=classifier_,
        preprocessing=preprocessing_,
        algo=tpe.suggest,
        max_evals = 30,
        trial_timeout=300)
    
    print("AutoML estimator in training...")
    t0 = time.time()
    estim.fit(train_data, train_target, cv_shuffle = False)
    estim_fit = time.time() - t0
    print("Best classifier selected and fitted in %.3f s\n" % estim_fit)
    
    print("Scoring: " + estim.score(test_data, test_target))
    print("Best selected model: " + estim.best_model())
    
    
def automl2(X_train, y_train, X_test, y_test):
    print('Init AutoML')
    est = HyperoptEstimator()
    print('Fit AutoML')
    est.fit(X_train, y_train)
    model = est.best_model()
    print(f'AutoML score: {est.score(X_test, y_test)}')
    return est

In [1]:
# Plots

def boxplots(data):
    for col in data.select_dtypes(['int64', 'float64']).columns:
        fig, ax = plt.subplots()
        bp = ax.boxplot(data[[col]].values)
        ax.set_xlabel(col)
        
def histograms(data):
    for col in data.select_dtypes(['object']).columns:
        fig, ax = plt.subplots()
        hg = ax.hist(data[[col]].values)
        ax.set_xlabel(col)
        
def correlation_matrix(df, labels):
    # sort by col names 
    r_df = df.reindex(labels, axis='columns')
    
    # create figure
    fig = plt.figure(0, figsize=(20,20))
    ax1 = fig.add_subplot(111)
    # get colormap
    cmap = cm.get_cmap('jet', 30)
    # show correlation matrix on 2D raster
    cax = ax1.imshow(r_df.corr(), interpolation="nearest", cmap=cmap)
    #ax1.grid(True)
    
    # add labels and ticks
    plt.title('Correlation')
    #ax1.set_xticks(range(r_df.shape[1]))
    #ax1.set_xticklabels(labels, fontsize=10, rotation='vertical')
    #ax1.set_yticks(range(r_df.shape[1]))
    #ax1.set_yticklabels(labels, fontsize=10)
    
    # Add colorbar, make sure to specify tick locations to match desired ticklabels
    fig.colorbar(cax, ticks=[-1, -.75, -.5, -.25, 0, .25, .5, .75, 1])
    plt.show()